In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmp61rsyo3q/pubring.gpg' created
gpg: /tmp/tmp61rsyo3q/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
cd drive

/content/drive


In [0]:
ls

class.txt
Code/
Colab Notebooks/
database/
Data Sets/
_detection reference/
drive/
Generating Enviornment Map (Reads)/
Google self driving car monthely report/
Initial tests/
Market Products and Raw Ideas/
Model_one.py
naya_folder_kuki naam nai sujh rha/
Our Product Draft/
pascal_data/
REFERENCES/
requirements.txt
Screen Shot 2018-07-02 at 8.38.24 PM.png
testModel.ipynb
tf1.png
Timeline/
Untitled document.odt


In [0]:
cd pascal_data

/content/drive/pascal_data


In [0]:
ls

ePascalVOCClassifier_epoch18above.h5       pascal_model_epoch20_.h5
ePascalVOCClassifier_epoch18aboveV_1_2.h5  pascalX.npy
ePascalVOCClassifier.h5                    pascalX_val.npy
ePascalVOCClassifier_monday_one.h5         pascaly.npy
pascal_model_epoch10.h5                    pascaly_val.npy


In [0]:
import numpy as np

X = np.load('pascalX.npy')
Y = np.load('pascaly.npy')

print(X.shape)
print(Y.shape)

(16551, 300, 300, 3)
(16551, 20)


In [0]:
X_val = np.load('pascalX_val.npy')
Y_val = np.load('pascaly_val.npy')

In [0]:
# import modules
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras import optimizers
import keras


Using TensorFlow backend.


In [0]:
x = Sequential()

# Block 1
x.add(Conv2D(64, (3, 3),activation='relu',padding='same',input_shape=(300,300,3)))
x.add(Conv2D(64, (3, 3),activation='relu',padding='same'))
x.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
x.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
x.add(Conv2D(128, (3, 3),activation='relu',padding='same',name='block2_conv2'))
x.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))
x.add(BatchNormalization())

# Block 3
x.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
x.add(Conv2D(128, (3, 3),activation='relu',padding='same'))

x.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))



# Classification block
x.add(Flatten())

# x.add(Dense(1000, activation='relu'))
x.add(Dense(800, activation='relu'))
x.add(Dense(20, activation='softmax', name='predictions'))




In [0]:
# original parameters (lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

# original parameters (lr=0.02, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
nadam = optimizers.Nadam(lr=0.02, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

# original parameters (lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

x.compile(loss='mean_squared_error',
              optimizer=adam,
              metrics=['accuracy'])

In [0]:
for i in range(25):
  print("epoch",i+1)
  x.fit(X, Y,epochs=1,batch_size=25,validation_data=(X_val, Y_val))
  x.save('littleClassifier.h5')
  print("----------------------------------------------------------------------")
  print("----------------------------------------------------------------------")


epoch 1
Train on 16551 samples, validate on 4952 samples
Epoch 1/1
  350/16551 [..............................] - ETA: 19:24 - loss: 0.1135 - acc: 0.1000